In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
mnist = tf.keras.datasets.mnist

(X_train, y_train),(X_test, y_test) = mnist.load_data()

print("The MNIST dataset has a training size of %d examples" %len(X_train))
print("The MNIST dataset has a test size of %d examples" %len(X_test))

The MNIST dataset has a training size of 60000 examples
The MNIST dataset has a test size of 10000 examples


In [3]:
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

print('X_train shaoe:', X_train.shape)
print(X_train.shape[0], 'train smaples')
print(X_test.shape[0], 'test smaples')


X_train shaoe: (60000, 28, 28)
60000 train smaples
10000 test smaples


In [4]:
from keras.utils import np_utils

num_classes = 10 
# print first ten (integer-valued) training labels
print('Integer-valued labels:')
print(y_train[:10])

# one-hot encode the labels
# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

# print first ten (one-hot) training labels
print('One-hot labels:')
print(y_train[:10])

Integer-valued labels:
[5 0 4 1 9 2 1 3 1 4]
One-hot labels:
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [5]:
img_rows, img_cols = 28, 28

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

print('input_shape: ', input_shape)
print('x_train shape:', X_train.shape)

input_shape:  (28, 28, 1)
x_train shape: (60000, 28, 28, 1)


In [6]:
## Model 1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D,BatchNormalization,Activation

# build the model object
model = Sequential()

# CONV_1: add CONV layer with RELU activation and depth = 32 kernels
model.add(Conv2D(8, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(BatchNormalization())

model.add(Conv2D(8, (3,3),  activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 
model.add(Conv2D(8, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

# POOL_1: downsample the image to choose the best features 
model.add(Conv2D(16, (3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Conv2D(16, (3,3), activation='relu'))
# model.add(Conv2D(8, (3,3) ,activation='relu'))
# model.add(Conv2D(16, (1,1) ,activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(8, (3,3), activation='relu'))
# model.add(Conv2D(8, (3,3), activation='relu'))
# # model.add(Conv2D(14, (3,3), activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.1))

model.add(Conv2D(10, 4, 4))                                          

# # Here we are Flateening our dat i.e making it one dimensional which we will feed to the network.
model.add(Flatten())


# # FC_2: output a softmax to squash the matrix into output probabilities for the 10 classes
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 8)         80        
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 8)         32        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 8)         584       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 8)         584       
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 10, 8)         32        
_________________________________________________________________
dropout (Dropout)            (None, 10, 10, 8)         0

In [7]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
              metrics=['accuracy'])


In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='model1.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
hist = model.fit(X_train, y_train, batch_size=128, epochs=40,
          validation_data=(X_test, y_test), callbacks=[checkpointer], 
          verbose=1, shuffle=True)

Epoch 1/40
469/469 [==============================] - 38s 69ms/step - loss: 0.2795 - accuracy: 0.9125 - val_loss: 0.1309 - val_accuracy: 0.9569

Epoch 00001: val_loss improved from inf to 0.13094, saving model to model1.weights.best.hdf5
Epoch 2/40
469/469 [==============================] - 28s 59ms/step - loss: 0.0736 - accuracy: 0.9777 - val_loss: 0.0559 - val_accuracy: 0.9824

Epoch 00002: val_loss improved from 0.13094 to 0.05587, saving model to model1.weights.best.hdf5
Epoch 3/40
469/469 [==============================] - 29s 61ms/step - loss: 0.0556 - accuracy: 0.9829 - val_loss: 0.0381 - val_accuracy: 0.9884

Epoch 00003: val_loss improved from 0.05587 to 0.03815, saving model to model1.weights.best.hdf5
Epoch 4/40
469/469 [==============================] - 29s 61ms/step - loss: 0.0470 - accuracy: 0.9849 - val_loss: 0.0452 - val_accuracy: 0.9864

Epoch 00004: val_loss did not improve from 0.03815
Epoch 5/40
469/469 [==============================] - 29s 62ms/step - loss: 0.0419